In [38]:
import torch

# Seleccionamos dispositivo CUDA si está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Usando dispositivo:", device)

Usando dispositivo: cuda


---

In [39]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

# Transformaciones básicas
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],
                         std =[0.229,0.224,0.225])
])

# Carga completa
dataset = datasets.ImageFolder(
    root=r"C:\Users\juanj\Desktop\Reconocimineto-AnimalesDomesticos-CNN-Explicabilidad\data\images",
    transform=transform
)
n = len(dataset)
n_train = int(n * 0.8)
n_val   = n - n_train

train_ds, val_ds = random_split(dataset, [n_train, n_val], 
                                generator=torch.Generator().manual_seed(42))

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=32, shuffle=False, num_workers=4, pin_memory=True)


In [40]:
num_classes = len(dataset.classes)  # ImageFolder guarda la lista de carpetas en .classes
print(f"Detectadas {num_classes} clases")

Detectadas 35 clases


In [41]:
import torch.nn as nn
from torchvision.models import resnet50

class FineTuneResNet50(nn.Module):
    def __init__(self, num_classes=37):
        super().__init__()
        self.backbone = resnet50(pretrained=True)
        # Congelamos todas las capas base
        for param in self.backbone.parameters():
            param.requires_grad = False

        # Reemplazamos la "fc" con nuestra cabeza:
        in_features = self.backbone.fc.in_features
        self.backbone.fc = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        return self.backbone(x)

model = FineTuneResNet50(num_classes=num_classes).to(device)


In [42]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.backbone.fc.parameters(), lr=1e-4)


In [ ]:
import torch, os
from tqdm import tqdm

# — Ya deberías tener: model, train_loader, val_loader, criterion, optimizer, device, dataset —

# 1) Si aún los necesitas, saca num_classes y class_to_idx del ImageFolder:
num_classes  = len(dataset.classes)
class_to_idx = dataset.class_to_idx

# 2) Funciones de entrenamiento/evaluación
def train_one_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss, total_correct = 0, 0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss   += loss.item() * images.size(0)
        total_correct+= (outputs.argmax(1) == labels).sum().item()
    return total_loss / len(loader.dataset), total_correct / len(loader.dataset)

def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss, total_correct = 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss   += loss.item() * images.size(0)
            total_correct+= (outputs.argmax(1) == labels).sum().item()
    return total_loss / len(loader.dataset), total_correct / len(loader.dataset)

# 3) Bucle de entrenamiento
num_epochs = 10
for epoch in range(1, num_epochs + 1):
    # Entrenamiento
    train_loop = tqdm(train_loader, desc=f"Epoch {epoch}/{num_epochs} [Train]", unit="batch")
    total_loss, total_correct = 0, 0
    model.train()
    for images, labels in train_loop:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss   += loss.item() * images.size(0)
        total_correct+= (outputs.argmax(1) == labels).sum().item()

        # Actualiza el postfix para ver loss y accuracy parcial
        avg_loss = total_loss / ((train_loop.n + 1) * train_loader.batch_size)
        avg_acc  = total_correct / ((train_loop.n + 1) * train_loader.batch_size)
        train_loop.set_postfix(loss=f"{avg_loss:.4f}", acc=f"{avg_acc:.2%}")

    # Validación
    val_loop = tqdm(val_loader, desc=f"Epoch {epoch}/{num_epochs} [Val]  ", unit="batch")
    total_loss, total_correct = 0, 0
    model.eval()
    with torch.no_grad():
        for images, labels in val_loop:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            total_loss   += loss.item() * images.size(0)
            total_correct+= (outputs.argmax(1) == labels).sum().item()

            avg_loss = total_loss / ((val_loop.n + 1) * val_loader.batch_size)
            avg_acc  = total_correct / ((val_loop.n + 1) * val_loader.batch_size)
            val_loop.set_postfix(loss=f"{avg_loss:.4f}", acc=f"{avg_acc:.2%}")

    # Al final de la época, imprime resumen
    train_loss = total_loss / len(train_loader.dataset)
    train_acc  = total_correct / len(train_loader.dataset)
    val_loss   = total_loss   / len(val_loader.dataset)
    val_acc    = total_correct / len(val_loader.dataset)
    print(f"→ Epoch {epoch:02d} | Train: {train_loss:.3f}, {train_acc:.2%} | Val: {val_loss:.3f}, {val_acc:.2%}")

# 4) Guardar checkpoint completo
ckpt = {
    "model_state_dict":     model.state_dict(),
    "optimizer_state_dict": optimizer.state_dict(),
    "num_classes":          num_classes,
    "class_to_idx":         class_to_idx,
    "transform": {
        "resize":            (224, 224),
        "normalize_mean":    [0.485, 0.456, 0.406],
        "normalize_std":     [0.229, 0.224, 0.225]
    },
    "split": {
        "seed":              42,
        "train_val_ratio":   0.8
    }
}

os.makedirs("checkpoints", exist_ok=True)
torch.save(ckpt, "checkpoints/resnet50_animals_exp.pth")
print("✅ Entrenamiento completado y checkpoint guardado en checkpoints/resnet50_animals_exp.pth")


Epoch 1/10 [Train]:   0%|          | 0/185 [00:00<?, ?batch/s]